# Demystifying ICP Purchasing Power Parity calculations: the Global Linking procedure

### Author: Giovanni Tonutti

---
Contents
- [Overview](#Overview) 
- [Load required Python libraries](#Libraries)  
- [Load input data](#InputData) 
- [Global Linking factors](#BHPPP)  
- [Above-basic heading PPPs](#aBHPPP)  
- [Aggregation through CAR-method](#CAR)

---

## Overview <a class="anchor" id="Overview"></a>
This notebook provides the accompanying code for the World Bank blog "Demystifying ICP Purchasing Power Parity calculations: the Global Linking procedure". Its purpose is to lay out the calculation steps and showcase the implementation of the main formulas needed to estimate ICP purchasing power parities (PPPs) at the global level. The blog post is publicly available [here](https://blogs.worldbank.org/opendata/demystifying-icp-purchasing-power-parity-calculations-using-python). 

*Note*: Because the target audience may include users unfamiliar with Python or programming in general, we opted to show each calculation step as explicitly as possible, at the cost of having more modularized and computationally efficient code.

## Load required Python libraries <a class="anchor" id="Libraries"></a>
The code will require loading the following well-known Python libaries: `pandas`, `numpy` and `statsmodels`

In [ ]:
## Load libraries 
import pandas as pd
import numpy as np 
import statsmodels.api as sm

## Load input data <a class="anchor" id="InputData"></a>

We start by loading the input datasets containing mock average price data, regional basic headings PPPs and other relevant country-level information. 

In [ ]:
#Load price data
data="price_data.csv"
prices=pd.read_csv(data) 
prices # Show full dataset 

In [ ]:
#Load ppp data
datappp="ppp_reg.csv"
ppp_reg=pd.read_csv(datappp) 
ppp_reg # Show full dataset 

This mock dataset contains 11 countries ('country'), each belonging to one of three different regions, and three basic headings ('bh'): garment; rice; and pork. ‘Basic headings’ in the ICP literature refer to detailed expenditure categories containing similar item varieties, for example the ‘Rice’ basic heading contains several rice varieties. It is also the lowest level of aggregation for which PPPs are first calculated. The different item varieties in each basic heading are noted under the ‘item’ column, for example, within ‘garment’ there are three item varieties, identified as ‘garment 1’, ‘garment 2’, and ‘garment 3’. Finally, an average price in the local currency unit of each country is reported for each item ('price') and information on the relative importance of each item in a country’s consumption at the basic heading level is included for each item priced in the importance column ('imp'). Following the guidelines provided by the [ICP Technical Advisory Group](https://www.worldbank.org/en/programs/icp#3), countries assign a weight of '3' to items identified as 'important' within a given basic heading and a weight of '1' to items deemed unimportant.

It should be highlighted that in practice the full [ICP classification](http://pubdocs.worldbank.org/en/708531575560035925/pdf/ICP-Classification-description-2019-1205.pdf) consists of 155 basic headings with the number of items within each varying from one basic heading to another. Also, not all countries are able to report prices for all items. These two realities are reflected in the example: some basic headings contain more items than others, and prices for some items are missing in some countries.


## Global linking factors <a class="anchor" id="BHPPP"></a>

Regions represent the first building block in the process of cross-country comparisons within the ICP framework. The Regional Implementing Agencies are responsible for the collection of price data and national accounts expenditures from countries and provide regional PPPs estimates. Each region designates one country within their region as numeraire and regional PPPs are calculated in relation to this regional numeraire.


Regional PPPs are then linked to the global numeraire via so-called ‘linking factors’. These are scalars estimated for each region via a regression method known as the ‘weighted region product dummy’ (RPD-W).
The RPD-W method is carried out within each basic heading by regressing the logarithm of the observed country item prices, converted into a common regional numéraire using the country's regional BH basic heading PPPs, on item dummies (one for each item) and region dummies (one for each region other than the region of the global numeraire). The RPD-W method also incorporates the country reported item-level importance indicators with the idea of ‘down-weighting’ less representative unrepresentative items during the calculation.

###  Select the base or numeraire currency 

The first step is to identify a global numeraire country. In our example, Country 11 acts as the global numeraire and region C as the reference region.

In [ ]:
## Select the base or numeraire currency
numeraire = 'C' 
numeraire_c = 'country11' 

###  Prep the inputs to run the RPD-W

In [ ]:
## Prep
## Drop country-item observations without a price
prices = prices[prices['price'].notnull()]

## Dataframe with country prices
d_region=pd.get_dummies(prices['region'])

## Prepare design matrix for CPD-W
d_region=pd.get_dummies(prices['region'])
d_region.drop(numeraire, axis=1, inplace=True) #drop numeraire
d_region = d_region.add_prefix('r_') #add prefix to countries
d_item=pd.get_dummies(prices['item'],drop_first=False) #include all item dummies
d_item = d_item.add_prefix('i_') #add prefix to items
prices=pd.concat([prices,d_region,d_item],axis=1) # Concatenate the new cols

## Create empty arrays to store results
l_coef= [] # to store exp(beta_hats)
l_bh= [] # to store bh labels

prices

###  Run the RPD-W on each basic heading and store results

In [ ]:
for bh in prices.bh.unique():
    tempdf=prices[prices.bh == bh] 
    X=tempdf.loc[:, [x for x in tempdf.columns if x.startswith(('r_', 'i_'))]]
    y = np.log(tempdf['price']/tempdf['ppp_reg']) 
    wts=tempdf['imp']

    wts_cpd=sm.WLS(y, X,weights=wts)
    res=wts_cpd.fit()
    res_eparams=np.exp(res.params)
    
    print("\n","Basic Heading:", bh, "\n")
    print('Exponentiated Parameters:',"\n",
          res_eparams)
    
    l_coef.append(res_eparams)
    l_bh.append(bh)

coef = np.array(l_coef, dtype=float)
cols = list(X)
 #store column heads of X as a list
coef[coef == 1] = np.nan #%% replace PPPs that were exp(0)=1 with 'np.nan'

#ppp=np.array(ppp_reg.ppp_reg, dtype=float)

The results above show the estimated coefficients from the RPD-W method for each of the three basic headings. Of particular interest are the estimated coefficients on the region dummies (denoted by the prefix 'r_') as they are the natural log of the estimated regions linking factors for the regional basic heading PPPs in question. Note that the estimated coefficients have already been exponentiated.

###  Gather and display the estimated LFs 

In [ ]:
#Create dataframe of PPP results from numpy arrays
#dimension = "# BHs" x "# coef"
df_bhppp=pd.DataFrame(data = coef, index = l_bh, columns = cols)
r_numeraire=f"r_{numeraire}"
df_bhppp.insert(0, r_numeraire, 1.000) #insert column of 1s for numeraire

df_bhppp=df_bhppp.loc[:, [x for x in df_bhppp.columns if x.startswith(('r_'))]] #subsetting to store only country level PPPs
df_bhppp.columns = df_bhppp.columns.str.replace('^r_', '') 

df_bhppp['bh'] = df_bhppp.index

df_bhppp=df_bhppp.melt(id_vars="bh",var_name="region", value_name="lf")
df_bhppp



## Above-basic heading PPPs <a class="anchor" id="aBHPPP"></a>

Next, regional PPPs are linked across regions through the estimated linking factirs at the basic heading level and successively aggregated using national accounts expenditure converted in regional PPPs for each country as weights.

The aggregation method involves constructing bilateral PPPs for each pair of countries, using basic heading-level national accounts expenditure values as weights from each country in turn. First, a Laspeyres-type bilateral PPP is calculated between each pair of countries and then a Paasche-type bilateral PPP. The geometric mean of the Laspeyres- and Paasche-type bilateral PPPs gives us the Fisher-type bilateral PPP between each pair of countries in the dataset. 

### Linking regional basic heading PPPs across regions

In [ ]:
#Merging the PPPreg data with the estimated Linking Factors from the RPD-W procedure
ppp_regLF=pd.merge(ppp_reg, df_bhppp, how='inner', on=('bh', 'region'))
ppp_regLF['ppp_linked']=ppp_regLF['ppp_reg']*ppp_regLF['lf']
ppp_regLF = ppp_regLF.drop(['ppp_reg'], axis=1)
ppp_regLF=ppp_regLF.pivot(index="bh",
             columns="country",
              values="ppp_linked").reset_index()


ppp_regLF.set_index(ppp_regLF['bh'], drop=True, append=False, inplace=True)
ppp_regLF = ppp_regLF.drop(['bh'], axis=1)
#Column sorting function
def sorting(first_col, df):
    columns = df.columns.tolist()
    columns.remove(first_col)
    columns.insert(0,first_col)
    return df.reindex(columns, axis=1)

#Sort cols with numeraire as col1
ppp_regLF=sorting(numeraire_c,ppp_regLF)
ppp_regLF

###  Load and display basic heading expenditure values
As a second step in the aggregation process, we load the basic heading level national accounts expenditure values in local currency unit for each country.

In [ ]:
#Load basic heading expenditure values
#Should contain bh and countries with prefix c
code="bhdata_exp.csv"
df_bh=pd.read_csv(code,index_col="icp_bh")
df_bh

In [ ]:
#sort rows alphabetically 
df_bh=df_bh.sort_values('icp_bh')

print("\n","Basic Heading Expenditure Values in Local Currency Units","\n")
print(df_bh, "\n")

###  Check the basic heading PPP and basic heading expenditure matrices
Before proceeding, it is important to check that both the basic heading PPP and basic heading expenditure matrices have the same dimensions. It is also important to check that the matrix of basic heading PPPs is complete. If the dimensions of the two matrices do not match or the basic heading PPP matrix is incomplete then aggregation at higher aggregate levels is not possible using the formulas employed by the ICP. 

In [ ]:
df_bh.columns = df_bh.columns.str.replace('^c_', '') 

print("Dimensions of Matrices (No. of headings x No. of countries):","\n")
print("BH Purchasing Power Parities (PPPs)  = ",ppp_regLF.shape)
print("BH Nominal Expenditures in LCUs      = ", df_bh.shape)

###  Calculate bilateral PPPs (Laspeyres-, Paasche-, and Fisher-type)

Calculate the Laspeyres-type bilateral PPPs

In [ ]:
#Calculate Laspeyres bilateral PPPs 
shape = (len(df_bh.columns),len(df_bh.columns))
lp = np.zeros(shape) #square matrix: country x country
nrow= len(lp)  # gets the number of rows
ncol = len(lp[0]) #get the number of cols

for row in range(nrow):
    for col in range(ncol):
        #weighted means by looping over df rows
        lp[row][col]= np.average((ppp_regLF.iloc[:,row]/ppp_regLF.iloc[:,col]),weights=df_bh.iloc[:,col])

lp_ppp = lp
lp_ppp=pd.DataFrame(data = lp_ppp, index = df_bh.columns, columns = df_bh.columns)

Square ('country x country') matrix of Laspeyres-type (bilateral) PPPs

In [ ]:
print("\n", "Laspeyres-type bilateral PPPs:","\n")
print(lp_ppp, "\n")

Derive the Paasche-type bilateral PPPs by taking the reciprocal of the transpose of the Laspeyres-type bilateral PPP 

In [ ]:
#Calculate Paasche bilateral PPPs 
pa = np.transpose(np.reciprocal(lp))
pa_ppp=pd.DataFrame(data = pa, index = df_bh.columns, columns = df_bh.columns)

Square ('country x country') matrix of Paasche-type (bilateral) PPPs

In [ ]:
print("\n", "Paasche-type bilateral PPPs:","\n")
print(pa_ppp, "\n")

Derive the Fisher-type bilateral PPPs by taking the geometric mean of the Laspeyres-type 
and Paasche-type bilateral PPPs for the aggregate

In [ ]:
#Create geomean function
def nangmean(arr, axis=None):
    arr = np.asarray(arr)
    inverse_valids = 1. / np.sum(~np.isnan(arr), axis=axis)  # could be a problem for all-nan-axis
    rhs = inverse_valids * np.nansum(np.log(arr), axis=axis)
    return np.exp(rhs)

#Calculate Fisher bilateral PPPs 
fi = np.zeros(shape)
nrow=len(fi)
ncol=len(fi[0])

for row in range(nrow):
    for col in range(ncol):
        fi[row][col]= nangmean([lp[row][col],pa[row][col]])

fi_ppp=pd.DataFrame(data = fi, index = df_bh.columns, columns = df_bh.columns)

Square ('country x country') matrix of Paasche-type (bilateral) PPPs

In [ ]:
print("Fisher-type bilateral PPPs:","\n")
print(fi_ppp, "\n")

###  Calculate GEKS PPPs

As a next step, the Gini-Éltető-Köves-Szulc (GEKS) method is applied to the matrix of Fisher-type bilateral PPPs. GEKS PPPs are calculated between each country relative to the numeraire or base country. To this end, the first step is to divide each country row of the Fisher-type bilateral PPP matrix by the row of the numeraire country. Each row will then contain two direct PPPs (each country to itself and directly to the numeraire country) and n−2 indirect PPPs (each country to the numeraire country via each of the other third countries), where n equals the total number of countries in the dataset. Finally, the GEKS PPP for each country relative to the numeraire is given by the geometric mean of the direct and indirect PPPs in each respective country row. 

GEKS PPPs are considered 'multilateral' because the GEKS procedure uses both direct and indirect PPPs and thus takes into account the relative prices between all the countries as a group. The GEKS method is needed to make the Fisher-type bilateral PPPs transitive and base country-invariant. Transitivity means that the PPP between any two countries should be the same whether it is computed directly or indirectly through a third country. Base country-invariant means that the PPPs between any two countries should be the same regardless of the choice of base or numeraire country.

In [ ]:
#Calculate GEKS multilateral ppps 
##requires the earlier nangmean function 
geks = np.zeros(shape)  # zero 'country x country' matrix
nrow=len(geks)  # gets the number of rows
ncol=len(geks[0])

for row in range(nrow):
    for col in range(ncol):
        geks[row][col]= nangmean(fi[row]/fi[col])     

geks_vec = np.zeros(shape=(1,len(df_bh.columns))) # as we need a vector of ppps, not a matrix
j=len(geks_vec[0])
for col in range(j):#..one PPP per country, or col of bhexp df
    geks_vec[:,col]=nangmean(geks[col,0]/geks[0,0]) #geomean over each row, w/ each col rebased to country in col1    

geks_ppp = np.array(geks_vec)

In [ ]:
geks_ppp = pd.DataFrame(geks_ppp)
geks_ppp.columns = df_bh.columns

print("\n","GEKS Multilateral PPPs:","\n")
print(geks_ppp.to_string(index=False), "\n")

## Aggregation throguh CAR-method <a class="anchor" id="CAR"></a>

The final step in the process of global PPPs estimation consists in the Country Aggregation with Redistribution (CAR) procedure. This step is undertaken to guarantee the principle of fixity. Fixity implies that the relative volumes in the global comparisons between any pair of countries belonging to a given region should be identical to the relative volumes of the two countries established in the regional comparisons to which they belong. 

In order to adhere to this principle, regional volume totals in the global comparison are obtained by summing the GEKS-adjusted volumes for individual countries in each region. These volume totals are then divided using the countries’ shares in regional comparison. Finally, PPPs in world numéraire for each country are derived indirectly by dividing countries' nominal expenditures by the volume-share adjusted expenditures. 

In [ ]:
#reshaping the global GEKS data frame 
geks_ppp=geks_ppp.melt(var_name="country", value_name="geks")
geks_ppp

In [ ]:
#Preparing a dataframe with aggregate regional PPPs and total expenditures 

volshare_df = ppp_reg[ppp_reg['bh'] == 'total']

df_bhtotal=df_bh.sum().to_frame().T 
df_bhtotal=df_bhtotal.melt(var_name="country", value_name="total_exp")

volshare_df=pd.merge(volshare_df, df_bhtotal, how='inner', on='country')
volshare_df['exp_ppp']=volshare_df['total_exp']/volshare_df['ppp_reg']
volshare_df['reg_total']= volshare_df['exp_ppp'].groupby(volshare_df['region']).transform('sum')
volshare_df['volshare']=volshare_df['exp_ppp']/volshare_df['reg_total']
volshare_df

In [ ]:
#Merging the expenditure and global geks dataframe
car_df=pd.merge(volshare_df, geks_ppp, how='inner', on=('country'))
#Converting the total exp using the global geks
car_df['exp_gek']=car_df['total_exp']/car_df['geks']
#Calculating the total regional expenditure in geks adjusted units
car_df['exp_gek_reg']=car_df['exp_gek'].groupby(volshare_df['region']).transform('sum')
#Applying the regional volume share to the total expenditure and re-basingit on the numeraire PPP
car_df['exp_adj']=car_df['exp_gek_reg']*car_df['volshare']
car_df['PPPglobal']=car_df['total_exp']/car_df['exp_adj']
car_df.set_index(car_df['country'], drop=True, append=False, inplace=True)
car_df
car_df['PPPglobal_num']=car_df['PPPglobal']/car_df.at['country11','PPPglobal']

print("\n","Global linked PPPs:","\n")
print(car_df.PPPglobal_num, "\n")

In the above example we showcased the main steps to calculate PPPs.  Information about the overall ICP methodology is provided on the [ICP website](https://www.worldbank.org/en/programs/icp/brief/methodology-calculation). 